# 1. Load Data

Load sales data from S3 / HDFS. We use the built-in "csv" method, which can use the first line has column names and which also supports infering the schema automatically. We use both and save some code for specifying the schema explictly.

We also peek inside the data by retrieving the first five records.

In [ ]:
from pyspark.sql.functions import *

data = spark.read\
    .option("header","true")\
    .option("inferSchema","true")\
    .csv("s3://dimajix-training/data/kc-house-data")

data.limit(5).toPandas()

## 1.1 Inspect Schema

Now that we have loaded the data and that the schema was inferred automatically, let's inspect it.

In [ ]:
# Print the schema of raw_data
# YOUR CODE HERE

# 2 Initial Investigations

As a first step to get an idea of our data, we create some simple visualizations. We use the Python matplot lib package for creating simple two-dimensional plots, where the x axis will be one of the provided attributes and the y axis will be the house price.

In [ ]:
%matplotlib inline

In [ ]:
# Import relevant Python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 2.1 House Price in Relation to sqft_living

Probably one of the most important attributes is the size of the house. This is provided in the data in the column "sqft_living". We extract the price column and the sqft_living column and create a simple scatter plot.

In [ ]:
# Extract price and one of the attributes
price = data.select("price").toPandas()
sqft_living = data.select("sqft_living").toPandas()

# Create simple scatter plot
plt.plot(sqft_living, price, ".")

## 2.2 House Price in Relation to sqft_lot

Another interesting attribute for predicting the house price might be the size of the whole lot, which is provided in the column "sqft_lot". So let's create another plot, now with "price" and "sqft_lot".

In [ ]:
price = data.select("price").toPandas()
sqft_lot = data.select("sqft_lot").toPandas()

plt.plot(sqft_lot, price, ".")

## 2.3 House Price in Relation to bathrooms

In [ ]:
price = data.select("price").toPandas()
sqft_lot = data.select("bathrooms").toPandas()

plt.plot(sqft_lot, price, ".")

# 3 Build Simple Linear Model

Now since there seems to be some relation between the house price and some attributes, we try to fit a linear model to our data. This will be performed in multiple small steps. Later on we will see a more concise syntax for specifying all steps.

## 3.1 Extract features by using VectorAssembler

Most Spark ML methods require one feature column of type `Vector`. In order to generate this feature column from the raw data, Spark provides a `VectorAssembler` which assembles one feature column from arbitrary source columns. The source columns have to be of type `double`.

We use it to automatically extract the columns

    bedrooms, bathrooms, sqft_living, sqft_lot, sqft_above, sqft_basement, sqft_living15, sqft_lot15

into the new output column 'features'

In [ ]:
from pyspark.ml.feature import *

cols = [
        'bedrooms',
        'bathrooms',
        'sqft_living',
        'sqft_lot',
        'sqft_above',
        'sqft_basement',
        'sqft_living15',
        'sqft_lot15'
    ]

# Build a VectorAssembler using the columns above
tx = # YOUR CODE HERE

# Transform the data
td = # YOUR CODE HERE

# Inspect the resulting schema
td.printSchema()

Let us have a look at the data

In [ ]:
td.limit(5).toPandas()

## 3.2 Split training / validation set

First we need to split the data into a training and a validation set. Spark already provides a DataFrame method called `randomSplit` which takes an array of weights (between 0 and 1) and creates as many subsets. In our example, we want to create a training data set with 80% and the validation set should contain the remaining 20%.

In [ ]:
# Split the data - 80% for training, 20% for validation
# YOUR CODE HERE

print("training_data = " + str(training_data.count()))
print("validation_data = " + str(validation_data.count()))

## 3.3 Build Model

Now that we have split up our data, we can fit a model to the training data. But before doing so, we again need to apply the `VectorAssembler` to the data to extract the features column

In [ ]:
from pyspark.ml.regression import *

# Create a LinearRegession algorithm and configure it to match our data
regression = # YOUR CODE HERE

# Train a linear model using the regression above
model = # YOUR CODE HERE

## 3.4 Predict

Make predictions from test data and print some results. We use the `validation_data` DataFrame (which was not used during training). Since this DataFrame does not already contain the feature column, we also need to apply the previously configured `VectorAssembler`.

In [ ]:
# Create features using the VectorAssembler
validation_features = # YOUR CODE HERE

# Transform the resulting DataFrame using the trained model
prediction = # YOUR CODE HERE

# Print result
prediction.limit(10).toPandas()

## 3.5 Evaluation

Finally lets evaluate the prediction. The less the difference between the real value and the predicted value, the better our model performs. But of course we need a definition of what is *near*. PySpark already provides some simple built-in metrics for evaluation.

In [ ]:
from pyspark.ml.evaluation import *

# YOUR CODE HERE

## 4 Adding more Features

The RMSE tells us that on average our prediction actually performs pretty bad. How can we improve that? Obviously we used only the size of the house for the price prediction so far, but we have a whole lot of additional information. So let's make use of that. The mathematical idea is that we create a more complex (but still linear) model that also includes other features.

Let's recall that a linear  model looks as follows:

    y = SUM(coeff[i]*x[i]) + intercept
    
This means that we are not limited to single feature `x`, but we can use many features `x[0]...x[n]`. Let's do that with the house data!

### Inspect data

Since we don't have any additional information, we model some of the features differently. So far we used all features as direct linear predictors, which implies that a grade of 4 is twice as good as 2. Maybe that is not the case and not all predictors have a linear influence. Specifically nominal and ordinal features should be modeled differntly as categories. More an that later.

First let's have a look at the data agin using Spark `describe`

In [ ]:
data.describe().toPandas()

Additionally let's check how many different zip codes are present in the data. If they are not too many, we could consider creating a one-hot encoded feature from the zip codes. We use the SQL function `countDistinct` to find the number of different zip codes.

In [ ]:
# Count the number of distinct ZIP Codes using the SQL function countDistinct
# YOUR CODE HERE

## 4.1 New Features using One-Hot Encoding

A simple but powerful method for creating new features from categories (i.e. nominal and ordinal features) is to use One-Hot-Encoding. For each nominal feature, the set of all possible values is indexed from 0 to some n. But since it cannot be assumed that larger values for n have a larger impact, a different approach is chosen. Instead each possible values is encoded by a 0/1 vector with only a single entry being one.

Lets try that with the tools Spark provides to us.

### Indexing Nominal Data
First we need to index the data. Since Spark cannot know, which or how many distinct values are present in a specific column, the `StringIndexer` works like a ML algorithm: First it needs to be fit to the data, thereby returning an `StringIndexerModel` which then can be used for transforming data.

Let's perform both steps and let us look at the result

In [ ]:
from pyspark.ml.feature import *

indexer = StringIndexer() \
    .setInputCol("zipcode") \
    .setOutputCol("zipcode_idx") \
    .setHandleInvalid("keep")

# Create index model using the `fit` method
index_model = # YOUR CODE HERE

# Apply the index by using the `transform` method of the index model
indexed_zip_data = # YOUR CODE HERE

# Inspect the result
indexed_zip_data.limit(10).toPandas()

You can also inspect the mapping by accessing all labels of the model

In [ ]:
# YOUR CODE HERE

An alternative way of configuring the indexer is to specify all relevant parameters in its constructor as follows:

In [ ]:
indexer = StringIndexer(
    inputCol = "zipcode",
    outputCol = "zipcode_idx",
    handleInvalid = "keep")

### One-Hot-Encoder

Now we have a single number (the index of the value) in a new column `zipcode_idx`. But in order to use the information in a linear model, we need to create sparse vectors from this index with only exactly one `1`. This can be done with the `OneHotEncoder` transformer. This time no fitting is required, the class can be used directly with its `transform` method.

In [ ]:
encoder = OneHotEncoder() \
    .setInputCol("zipcode_idx") \
    .setOutputCol("zipcode_onehot")

encoded_zip_data = encoder.transform(indexed_zip_data)
encoded_zip_data.limit(10).toPandas()

## 4.2 Creating Pipelines

Since it would be tedious to add all features one after another and apply a full chain of transformations to the training set, the validation set and eventually to new data, Spark provides a `Pipeline` abstraction. A Pipeline simply contains a sequence of Transformations and (possibly multiple) machine learning algorithms. The whole pipeline then can be trained using the `fit` method which will return a `PipelineModel` instance. This instance contains all transformers and trained models and then can be used directly for prediction.

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.ml.regression import *

pipeline = Pipeline(stages = [
    # We use the SQLTransformer to calculate the absolute age and the age since the last renovation
    # Store both ages in "abs_age" and "renovated_age"

    # For every nominal feature, you have to create a pair of StringIndexer and OneHotEncoder. 
    # The StringIndexer should store its index result in some new column, which then is used 
    # by the OneHotEncoder to create a one-hot vector.
    StringIndexer(
        inputCol = "bathrooms",
        outputCol = "bathrooms_idx",
        handleInvalid = "keep"),
    OneHotEncoder(
        inputCol = "bathrooms_idx",
        outputCol = "bathrooms_onehot"),
    # Add StringIndexers and OneHotEncoders for the following nominal columns:
    # "bedrooms", "floors", "grade", "zipcode"
    # YOUR CODE HERE
    
    # In addition add OneHotEncoder for the columns "view" and "condition"
    # YOUR CODE HERE
    
    # Now add a VectorAssembler which collects all newly derived columns from above and the following numeric columns:
    # "sqft_living", "sqft_lot", "waterfront", "sqft_above", "sqft_basement", "yr_built", "yr_renovated", "sqft_living15", "sqft_lot15"
    # YOUR CODE HERE
    
    # Finally add a LinearRegression which uses the output of the VectorAssembler as features and the
    # target variable "price" as label column
    # YOUR CODE HERE

    ]
)


### Train model with training data

Once you created the `Pipeline`, you can fit it in a single step using the `fit` method. This will return an instance of the class `PipelineModel`. Assign this model instace to a value called `model`.

And remember: Use the training data for fitting!

In [ ]:
model = # YOUR CODE HERE

## 4.3 Evaluate model using validation data

Now that we have a model, we need to measure its performance. This requires that predictions are created by applying the model to the validation data by using the `transform` method of the moodel. The quality metric of the prediction is implemented in the `RegressionEvaluator` class from the Spark ML evaluation package. Create an instance of the evaluator and configure it appropriately to use the column `price` as the target (label) variable and the column `prediction` (which has been created by the pipeline model) as the prediction column. Also remember to set the metric name to `rmse`. Finally feed in the predicted data into the evaluator, which in turn will calculate the desired quality metric (RMSE in our case).

In [ ]:
from pyspark.ml.evaluation import *

# Create and configure a RegressionEvaluator
evaluator = # YOUR CODE HERE

# Create predictions of the validationData by using the "transform" method of the model
pred = # YOUR CODE HERE

# Now measure the quality of the prediction by using the "evaluate" method of the evaluator
rmse = # YOUR CODE HERE

print("RMSE = " + str(rmse))